In [1]:
# Day_03_02_gan.py
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
import os

In [2]:
mnist = input_data.read_data_sets('mnist', one_hot=True)

ph_x = tf.placeholder(tf.float32, shape=[None, 784])
ph_z = tf.placeholder(tf.float32, shape=[None, 128])

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting mnist\t10k-images-idx3-ubyte.gz
Extracting mnist\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
def generator(noises):
    with tf.variable_scope('generator'):
        hidden = tf.layers.dense(noises, 256, tf.nn.relu)
        output = tf.layers.dense(hidden, 784, tf.nn.sigmoid)

    return output

In [4]:
# reuse는 변수를 다시 갖다 쓴다는 개념
def discriminator(inputs, reuse=None):
    with tf.variable_scope('discriminator') as scope:
        if reuse:
            scope.reuse_variables()

        hidden = tf.layers.dense(inputs, 256, tf.nn.relu)
        output = tf.layers.dense(hidden, 784, tf.nn.sigmoid)

    return output

In [5]:
def get_noises(batch_size, n_noise):
    # 난수를 만드는 함수
    return np.float32(np.random.normal(size=[batch_size, n_noise]))

In [6]:
K = generator(ph_z)
D = discriminator(ph_x)
G = discriminator(K, reuse=True)

D_loss = tf.reduce_mean(tf.log(D) + tf.log(1-G))
G_loss = tf.reduce_mean(tf.log(G))

D_opt = tf.train.AdamOptimizer(0.0002)
G_opt = tf.train.AdamOptimizer(0.0002)

D_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='discriminator')
G_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='generator')

D_train = D_opt.minimize(-D_loss, var_list=D_vars)
G_train = G_opt.minimize(-G_loss, var_list=G_vars)

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

epochs = 30
batch_size = 100  # 출구를 몇개씩 조사할건지
n_iters = mnist.train.num_examples // batch_size  # 550

samples = []
sample_count = 7
# epcochs를 1번 돌고 념
for i in range(epochs):
    D_cost, G_cost = 0, 0
    for j in range(n_iters):
        xx, _ = mnist.train.next_batch(batch_size)
        noises = get_noises(batch_size, 128)

        _, c1 = sess.run([D_train, D_loss], feed_dict={ph_x: xx, ph_z:noises})
        _, c2 = sess.run([G_train, G_loss], feed_dict={ph_z: noises})

        D_cost += c1
        G_cost += c2

    print('[{}] {:5.3f} : {:5.3f}'.format(i, D_cost / n_iters, G_cost /n_iters))

    noises = get_noises(sample_count, 128)
    samples.append(sess.run(K, {ph_z: noises}))

In [ ]:
_, ax = plt.subplots(epochs, sample_count, figsize=(sample_count, epochs))

In [ ]:
for i in range(epochs):
    for j in range(sample_count):
        ax[i, j].set_axis_off()
        ax[i, j].imshow(np.reshape(samples[i][j], [28, 28]), cmap='gray')

plt.tight_layout()
plt.show()

In [ ]:
sess.close()